In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import re
from nltk.corpus import stopwords

In [4]:
df = pd.read_csv(r"E:\Users\User\Files\Articles.csv")
df.head()

,article_id,article_text
0,1,The CIA managed to get most of its Afghan info...
1,2,Consider the amount of plastic you put into th...


In [5]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [10]:
!pip install glove-python-binary

In [12]:
pip install -U mittens

Note: you may need to restart the kernel to use updated packages.


In [7]:
from mittens import GloVe

In [16]:
word_embeddings = {}
f = open(r"E:\Users\User\Files\glove.6B(1)\glove.6B.100d.txt", encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

<ipython-input-16-1f27de05fd8d>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [23]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [24]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
print(len(sentence_vectors))
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

77


In [25]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [26]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(2):
  print("ARTICLE:")
  print(df['article_text'][i])
  print('\n')
  print("SUMMARY:")
  print(ranked_sentences[i][1])
  print('\n')

ARTICLE:
The CIA managed to get most of its Afghan informants and spies out of the country ahead of the U.S. pullout this week, according to two sources familiar with the agency�s operations, even as the State Department admitted that the majority of Afghans who worked for the United States over the past two decades�interpreters and others�were left behind.��We got everybody, more or less, out. It�s thousands of people,� including family members of the spies, a Congressional official told Foreign Policy, requesting anonymity to discuss sensitive matters. �They are the ones the Taliban could have identified.� 
Another source familiar with the CIA�s evacuation efforts said the agency�s Special Activities Center, which is staffed in part by elite paramilitary forces and runs covert operations, took part in the evacuation. The operations took place across the country, the source said�not just around the Afghan capital of Kabul, where many thousands of people desperate to flee the country c

KeyError: 2

In [27]:
print(ranked_sentences)

[(0.014469310191078976, '�Wishcycling,� or tossing every type of plastic into the recycling bin and hoping for the best, can make separating out useful material more difficult and actually reduce the amount of plastic that is recycled, says Jeff Donlevy, the general manager at a California recycling facility who has been in the industry for more than 25 years.'), (0.014438720789902841, 'Experts say that while cutting back on plastic use is a worthy individual goal (see �How to Quit Plastic�), the only way to stem the rising tide of plastic is for companies to make less of it and for recycling programs to be retooled so that more of what we throw away is actually turned into something useful.'), (0.014303377757670642, 'Many of these plastic items are useful and convenient, but they also come with a high environmental cost.'), (0.014215348161767479, 'Plastic production is expected to more than double by 2050, and even if it doesn�t, the plastic trash that people continue to throw away wi